# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 11:16AM,244477,10,SONSB0001901,"Nextsource Biotechnology, LLC",Released
1,Aug 18 2022 11:16AM,244477,10,SONSB0001902,"Nextsource Biotechnology, LLC",Released
2,Aug 18 2022 11:16AM,244477,10,SONSB0001903,"Nextsource Biotechnology, LLC",Released
3,Aug 18 2022 11:16AM,244475,15,8377476,"Mizner Bioscience, LLC",Released
4,Aug 18 2022 11:16AM,244475,15,8414883,"Mizner Bioscience, LLC",Released
5,Aug 18 2022 11:16AM,244475,15,8414884,"Mizner Bioscience, LLC",Released
6,Aug 18 2022 11:16AM,244475,15,8414885,"Mizner Bioscience, LLC",Released
7,Aug 18 2022 11:15AM,244476,15,VT80176670,"Virtus Pharmaceuticals, LLC",Released
8,Aug 18 2022 11:15AM,244476,15,VT80176676,"Virtus Pharmaceuticals, LLC",Released
9,Aug 18 2022 11:15AM,244476,15,VT80176677,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,244470,Executing,1
51,244474,Released,1
52,244475,Released,4
53,244476,Released,7
54,244477,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244470,NaN,1.0,NaN
244474,NaN,NaN,1.0
244475,NaN,NaN,4.0
244476,NaN,NaN,7.0
244477,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18.0,2.0,0.0
244357,9.0,3.0,1.0
244385,11.0,4.0,0.0
244396,0.0,0.0,73.0
244405,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18,2,0
244357,9,3,1
244385,11,4,0
244396,0,0,73
244405,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,0
1,244357,9,3,1
2,244385,11,4,0
3,244396,0,0,73
4,244405,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,
1,244357,9,3,1
2,244385,11,4,
3,244396,,,73
4,244405,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC"
3,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC"
7,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC"
14,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc."
15,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc."
51,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation
52,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc."
55,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc."
69,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc."
70,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC",,,3
1,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC",,,4
2,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC",,,7
3,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc.",,,1
4,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",,,36
5,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,,1,
6,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",,,3
7,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc.",,,14
8,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",,,1
9,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC",3,,
1,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC",4,,
2,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC",7,,
3,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc.",1,,
4,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36,,
5,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,,1,
6,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",3,,
7,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc.",14,,
8,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1,,
9,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC",3,,
1,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC",4,,
2,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC",7,,
3,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc.",1,,
4,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
1,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC",4.0,NaN,NaN
2,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC",7.0,NaN,NaN
3,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 11:16AM,244477,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
1,Aug 18 2022 11:16AM,244475,15,"Mizner Bioscience, LLC",4.0,0.0,0.0
2,Aug 18 2022 11:15AM,244476,15,"Virtus Pharmaceuticals, LLC",7.0,0.0,0.0
3,Aug 18 2022 11:01AM,244474,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4400050,144.0,12.0,9.0
102,244450,1.0,0.0,0.0
12,488870,1.0,14.0,0.0
15,733415,47.0,0.0,0.0
16,733287,21.0,0.0,0.0
17,488861,2.0,0.0,0.0
19,733234,19.0,12.0,18.0
20,1466526,95.0,14.0,18.0
21,1466560,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4400050,144.0,12.0,9.0
1,102,244450,1.0,0.0,0.0
2,12,488870,1.0,14.0,0.0
3,15,733415,47.0,0.0,0.0
4,16,733287,21.0,0.0,0.0
5,17,488861,2.0,0.0,0.0
6,19,733234,19.0,12.0,18.0
7,20,1466526,95.0,14.0,18.0
8,21,1466560,6.0,0.0,0.0
9,22,488943,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,144.0,12.0,9.0
1,102,1.0,0.0,0.0
2,12,1.0,14.0,0.0
3,15,47.0,0.0,0.0
4,16,21.0,0.0,0.0
5,17,2.0,0.0,0.0
6,19,19.0,12.0,18.0
7,20,95.0,14.0,18.0
8,21,6.0,0.0,0.0
9,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,144.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,47.0
4,16,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21,22
Status,,,,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
Executing,12.0,0.0,14.0,0.0,0.0,0.0,12.0,14.0,0.0,0.0
Released,144.0,1.0,1.0,47.0,21.0,2.0,19.0,95.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,12.0,0.0,14.0,0.0,0.0,0.0,12.0,14.0,0.0,0.0
2,Released,144.0,1.0,1.0,47.0,21.0,2.0,19.0,95.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,12.0,0.0,14.0,0.0,0.0,0.0,12.0,14.0,0.0,0.0
2,Released,144.0,1.0,1.0,47.0,21.0,2.0,19.0,95.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()